In [36]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import ssl

## DBMS 코드에서 연결 사용
- DBMS 각각에 맞는 패키지가 설치되어야 함
- MySQL(MriadDB)은 pymysql 패키지 필요 (설치해야 됨)

In [37]:
!pip install pymysql

In [39]:
import pymysql

### pymysql 사용법
1. dbms 연결
    - pymysql.connect(host='서버주소(ip주소)', port=3306, user='userid', passwd='userpass', charset='utf8')
    - port : 서버주소를 이용해서 서버 컴퓨터의 입구까지 연결
        - port를 이용해서 각 프로그램의 방으로 연결
        
2. 1번에서 연결한 정보(객체)를 이용해서 cursor 객체 생성
    - connect.cursor()
    - cursor객체 : dbms와 통로 역할을 함
        - sql문의 명령을 dbms로 전달
        - 실행된 sql문의 결과 반환
        
3. sql 문 작성
    - 저장소(table) 관련 sql 문 : create, alter, drop 으로 시작되는 sql문
    - 데이터 저장 관련 sql 문 : table 데이터 삽입 삭제 갱신과 관련
        - insert, delete, update로 시작되는 sql 문
    - 검색 sql문 : select로 시작되는 sql문 (반환데이터가 있음)
    
4. cursor 객체를 이용해서 3번 작성한 sql문을 실행
    - cursor.execute(sql문)
    
5. 검색과 관련된 sql문인 경우
    - cursor.fetchXXX() 를 이용해서 데이터 반환
    - cursor.fetchall() : 반환된 모든 데이터 한번에 전달받기
    - cursor.fetchone() : 반환된 모든 데이터를 한개씩 전달받기

In [41]:
# connect 객체 생성
db = pymysql.connect(host='localhost', port=3306, user='root', passwd='root', charset='utf8')
# cursor 객체
cursor = db.cursor()

In [42]:
sql = "drop database beauty_shop"  # beauty_shop db 삭제할 것
# cursor.execute(sql)

In [43]:
# db 없어서 에러 발생
OperationalError: (1008, "Can't drop database 'beauty_shop'; database doesn't exist")

### dbms 구조
- 제일 넓은 구조 : database
    - 생성 : create database db 명 단, db 명은 중복되면 안됨
    - 삭제 : drop database db 명 단, db명은 기존에 생성되어 있었어야 함
- database 내에 실제 data 저장 구조 : table
    - 생성 : create table
    - 삭제 : drop table
    - 수정 : alter table

In [44]:
# db 생성
sql = 'create database beauty_shop'
cursor.execute(sql)

1

In [45]:
# 생성된 db 확인
sql = 'show databases' # 반환결과가 있는 구문
cursor.execute(sql)
result = cursor.fetchall() # dbmas의 db 명 모두 반환
result

(('beauty_shop',),
 ('information_schema',),
 ('mysql',),
 ('performance_schema',),
 ('sys',))

In [46]:
# 사용할 db를 결정해서 명령
# use db 명
sql = "use beauty_shop"
cursor.execute(sql)

0

In [47]:
# 현재 사용 db 반환
sql = 'select database()'  # 현재 설정된 db 명 반환
cursor.execute(sql)  # dbms로 sql 명령문 전달 후 결과 반환
result = cursor.fetchone()
result

('beauty_shop',)

In [48]:
# 쇼핑몰 크롤링 data를 저장할 table 생성(dbms에 생성-sql문 사용)
sql = '''
    CREATE TABLE product (
        PRODUCT_CODE int AUTO_INCREMENT NOT NULL,
        TITLE VARCHAR(200) NOT NULL,
        ORI_PRICE FLOAT,
        DISCOUNT_PRICE FLOAT,
        link VARCHAR(200),
        PRIMARY KEY(PRODUCT_CODE)
    );
'''

In [18]:
# 테이블 생성 sql문(쿼리문)실행하기
cursor.execute(sql)

# 외부에서 연결해서 사용할 경우 dbms(마리아db)는 연결을 지속시킬 수 없기 때문에
# 명령을 클라이언트 단에서 먼저 진행하고 나중에 dbms에 반영한다(create table insert 같은 명령)
# 테이블 같은 경우 생성하고 바로 명시적 반영 하는게 좋음
# connenct객체 함수 commit()을 사용

db.commit() # dbms에 완전 반영

In [19]:
# 테이블 생성 여부 확인
sql = "show tables"
cursor.execute(sql)
result=cursor.fetchall()
result

(('product',),)

In [20]:
sql = "desc product"  # desc 테이블명 : 테이블 컬럼 구조 반환
cursor.execute(sql)
result=cursor.fetchall()  # 여러 컬럼이므로 fetchall() 사용
result

(('PRODUCT_CODE', 'int(11)', 'NO', 'PRI', None, 'auto_increment'),
 ('TITLE', 'varchar(200)', 'NO', '', None, ''),
 ('ORI_PRICE', 'float', 'YES', '', None, ''),
 ('DISCOUNT_PRICE', 'float', 'YES', '', None, ''),
 ('link', 'varchar(200)', 'YES', '', None, ''))

### 제품 1개에 대한 정보 추출 후 db table에 저장

In [21]:
# df 생성
prd_dict = {'품목':[], '가격':[], '세일가격':[],'제품경로':[]}
prd_df = pd.DataFrame(prd_dict)
prd_df

,품목,가격,세일가격,제품경로


In [22]:
# 각 제품 정보 추출하는 함수
# box 는 class 값이 description인 div 태그 1개가 전달
def get_product_info(box) :
    try : 
        strong = box.find('strong',{'class':'name'})
        title= strong.text.split(":")[1].strip()
        link = strong.a['href']
        lis = box.find('ul').findAll('li')
        price=(lis[0].text.split(' ')[2])
        sale_price=(lis[1].text.split(' ')[2])
        return {'품목':title, '가격':price, '세일가격':sale_price,'제품경로':link}
    except : 
        print('에러')

In [23]:
# 전역변수 df 활용해서 제품 정보 저장
# 최초 호출함수
def get_page_product(url) :
    global prd_df 
    # global 변수를 함수 내부가 아닌 외부에서 찾을 것
    # global은 전역변수(함수내/외부에서 모두 사용)로 생성
    bs_obj = get_request_product(url)
    # 페이지 내에 전체 제품 정보 추출
    boxes = bs_obj.findAll('div',{'class':'description'})
    # 각 제품에 대한 정보 추출후 df에 저장
    for box in boxes[2:] : # best 제외
        prd = get_product_info(box)
        save_data(prd) # db에 저장하는 함수호출(사용자정의함수)
#         res = pd.DataFrame(get_product_info(box),index=range(1,2))
#         prd_df = pd.concat([prd_df,res],axis=0,ignore_index=True)

### db table에 data 저장
- Insert into 구문 사용
- insert into 테이블명(컬럼명1, 컬럼명2,...,컬럼명n) values (컬럼값1, 컬럼값2,... 컬럼값)
- 문자열값은 ''로 표현
- insert into product(title, ori_price, discount_price, link) values ('toner',22.0,17.0,'http://a.b.c')

In [24]:
insert into product(title, ori_price, discount_price, link) values 
('SKINFOOD Yuja C Dark Spot Clear Toner 200ml',39.00,35.10,'/product/skinfood-yuja-c-dark-spot-clear-toner-200ml/62935/category/1019/display/1/')

SyntaxError: invalid syntax (1632826842.py, line 1)

In [25]:
# insert 구문을 이용해서 save_data 함수 생성
# prd_info 라는 제품 1개의 정보가 딕셔너리로 전달 됨
def save_data(prd_info) :
    sql = "insert into product(title, ori_price, discount_price, link) values ('"
    sql +=  prd_info['품목'] + "'," 
    sql += prd_info['가격'] + "," 
    sql += prd_info['세일가격'] + ",'"
    sql += prd_info['제품경로'] + "')"
    print(sql)
    cursor.execute(sql)   

In [26]:
def get_request_product(url) : 
    # 요청 후 코드 추출
    url = url
    context = ssl._create_unverified_context()
    htmls = urlopen(url, context=context)
    htmls = htmls.read()
    # bs4 객체 생성
    bs_obj = BeautifulSoup(htmls,'html.parser')
    return bs_obj

In [27]:
url = 'https://jolse.com/category/toners-mists/1019?page=4'

In [28]:
# db 저장 진생 후 확인
get_page_product(url)

insert into product(title, ori_price, discount_price, link) values ('COSRX Hydrium Watery Toner 150ml',28.00,16.80,'/product/cosrx-hydrium-watery-toner-150ml/30829/category/1019/display/1/')
insert into product(title, ori_price, discount_price, link) values ('Nature Republic Soothing & Moisture Aloe Vera 92% Soothing Gel Mist 155ml',10.50,7.87,'/product/nature-republic-soothing-moisture-aloe-vera-92-soothing-gel-mist-155ml/43720/category/1019/display/1/')
insert into product(title, ori_price, discount_price, link) values ('ONE THING Artemisia Capillaris Extract 300ml',30.58,27.52,'/product/one-thing-artemisia-capillaris-extract-300ml/40703/category/1019/display/1/')
insert into product(title, ori_price, discount_price, link) values ('iUNIK Vitamin Hyaluronic Acid Vitalizing Toner 200ml',21.99,19.79,'/product/iunik-vitamin-hyaluronic-acid-vitalizing-toner-200ml/18489/category/1019/display/1/')
insert into product(title, ori_price, discount_price, link) values ('ETUDE Moistfull Collagen 

ProgrammingError: (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 'PIEU Raspberry Vinegar Hair Mist 105ml',16.07,14.46,'/product/apieu-raspberry...' at line 1")

In [29]:
db.commit() # dbms 반영

In [30]:
# db에서 data 조회 후 확인
sql = "select * from product" # product 테이블의 모든(*) data 반환
cursor.execute(sql)
result = cursor.fetchall()
result

((1,
  'COSRX Hydrium Watery Toner 150ml',
  28.0,
  16.8,
  '/product/cosrx-hydrium-watery-toner-150ml/30829/category/1019/display/1/'),
 (2,
  'Nature Republic Soothing & Moisture Aloe Vera 92% Soothing Gel Mist 155ml',
  10.5,
  7.87,
  '/product/nature-republic-soothing-moisture-aloe-vera-92-soothing-gel-mist-155ml/43720/category/1019/display/1/'),
 (3,
  'ONE THING Artemisia Capillaris Extract 300ml',
  30.58,
  27.52,
  '/product/one-thing-artemisia-capillaris-extract-300ml/40703/category/1019/display/1/'),
 (4,
  'iUNIK Vitamin Hyaluronic Acid Vitalizing Toner 200ml',
  21.99,
  19.79,
  '/product/iunik-vitamin-hyaluronic-acid-vitalizing-toner-200ml/18489/category/1019/display/1/'),
 (5,
  'ETUDE Moistfull Collagen Facial Toner 200ml',
  19.23,
  17.31,
  '/product/etude-moistfull-collagen-facial-toner-200ml/23478/category/1019/display/1/'),
 (6,
  'MANYO FACTORY Bifida Cica Herb Toner 210ml',
  22.0,
  19.8,
  '/product/manyo-factory-bifida-cica-herb-toner-210ml/42278/category/

In [31]:
# 사용 완료 후 db 닫기
db.close()

### 판다스 패키지 사용해서 db에 있는 data df로 load 하기
- pd.read_sql(sql문장, db connect 객체)

In [49]:
# connect 객체 생성
db = pymysql.connect(host='localhost',port=3306, user='root',passwd='root', charset='utf8')
# cursor 객체
cursor = db.cursor()

In [50]:
sql = "use beauty_shop"
cursor.execute(sql)

0

In [51]:
sql = "select * from product"
df = pd.read_sql(sql, db)
df.head()

C:\Users\사용자\AppData\Local\Temp\ipykernel_22396\1301940393.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, db)


DatabaseError: Execution failed on sql 'select * from product': (1146, "Table 'beauty_shop.product' doesn't exist")